In [ ]:
! python3 setup.py install

# Gaussian PSF 

In [ ]:
from deltamic import Gaussian_psf, compute_spatial_frequency_grid,normalize_tensor,render_image_from_ftmesh,Fourier3dMesh,compute_box_size
import trimesh
import numpy as np
import torch
import skimage.io as io 
from skimage import img_as_ubyte
from time import time

"""
Parameters:
"""

if torch.cuda.is_available():
    print("Computations done in CUDA!")
    device = 'cuda:0'
    n_subdivisions = 2
    box_shape = np.array([100]*3)
else : 
    print("No CUDA devices available, Computations done in CPU")
    device = 'cpu'
    n_subdivisions = 1
    box_shape = np.array([60]*3).tolist()

"""
Creation of
initial 
configurations: 
"""

mesh = trimesh.primitives.Sphere(subdivisions = n_subdivisions)
verts, faces = np.array(mesh.vertices),np.array(mesh.faces)
verts-=verts.min(axis=0)
verts*=0.5
box_size = compute_box_size(verts, offset = 0.35)
Verts = torch.tensor(verts, dtype = torch.float, device = device,requires_grad = True)
Faces = torch.tensor(faces, dtype = torch.long, device = device)
Faces_coeff = torch.ones(len(Faces),dtype = torch.float, device = device)
box_size = torch.tensor(box_size) 

print("Vertices shape: ",verts.shape,"Faces shape: ", faces.shape)
print("Min/Max x/y/z position of vertices: ",verts.min(axis=0),verts.max(axis=0))
print("box_size: ",box_size, "box_shape",box_shape)

narrowband_thresh = torch.tensor(0,dtype = torch.float, device = device)
meshFT = Fourier3dMesh(box_size,box_shape,device=device, dtype = torch.float32)

model = Gaussian_psf(box_shape,box_size[:,1],sigma = 1e3,device = device)
PSF = model.forward()
OTF = torch.abs(torch.fft.fftshift(torch.fft.fftn(PSF)))

"""
Image creation:
"""
from deltamic.optimize import VectorAdam

t1 = time()
ftmesh = meshFT(Verts,Faces, Faces_coeff)

image_ft=normalize_tensor(render_image_from_ftmesh(ftmesh, OTF, box_shape))
t2 = time()

io.imsave("Test_sphere_Gaussian.tif",img_as_ubyte(image_ft.detach().cpu().numpy()))
print("forward pass computed successfully in", t2-t1,"seconds", "Image saved as Test_sphere_Gaussian.tif")

loss = torch.sum(torch.abs(ftmesh))
t1 = time()
loss.backward()
t2 = time()
print("backward pass computed successfully in", t2-t1,"seconds")




# Pupil Function

In [ ]:
from deltamic import Hanser_psf, compute_spatial_frequency_grid,normalize_tensor,render_image_from_ftmesh,Fourier3dMesh,compute_box_size
import trimesh
import numpy as np
import torch
import skimage.io as io 
from skimage import img_as_ubyte
from time import time


"""
Parameters:
"""

if torch.cuda.is_available():
    print("Computations done in CUDA!")
    device = 'cuda:0'
    n_subdivisions = 2
    box_shape = np.array([100]*3)
else : 
    print("No CUDA devices available, Computations done in CPU")
    device = 'cpu'
    n_subdivisions = 1
    box_shape = np.array([60]*3).tolist()

"""
Creation of
initial 
configurations: 
"""

mesh = trimesh.primitives.Sphere(subdivisions = n_subdivisions)
verts, faces = np.array(mesh.vertices),np.array(mesh.faces)
verts-=verts.min(axis=0)
verts*=0.5
box_size = compute_box_size(verts, offset = 0.35)
Verts = torch.tensor(verts, dtype = torch.float, device = device,requires_grad = True)
Faces = torch.tensor(faces, dtype = torch.long, device = device)
Faces_coeff = torch.ones(len(Faces),dtype = torch.float, device = device)
box_size = torch.tensor(box_size) 

print("Vertices shape: ",verts.shape,"Faces shape: ", faces.shape)
print("Min/Max x/y/z position of vertices: ",verts.min(axis=0),verts.max(axis=0))
print("box_size: ",box_size, "box_shape",box_shape)

narrowband_thresh = torch.tensor(0,dtype = torch.float, device = device)


psf_params = dict(na=1.27, ni=1.33, wl=0.561, vec_corr="none")

# Set the Nyquist sampling rate
nyquist_sampling = psf_params["wl"] / psf_params["na"] /4

# our oversampling factor, must be odd for easy integration (for peaked symmetrical funtions the kernel must be odd)
oversample_factor = 1

psf_params["res"] = nyquist_sampling / oversample_factor

meshFT = Fourier3dMesh(box_size,box_shape,device=device, dtype = torch.float32)
model = Hanser_psf(box_shape,meshFT.xi0,meshFT.xi1,meshFT.xi2,NA=psf_params['na'], wavelength=psf_params['wl'], ni=psf_params['ni'], res_lateral=psf_params['res'],res_axial=psf_params['res'], num_zernike = 21, device = device)
PSF = model.forward()
OTF = torch.abs(torch.fft.fftshift(torch.fft.fftn(PSF)))

"""
Image creation:
"""
from deltamic.optimize import VectorAdam

t1 = time()
ftmesh = meshFT(Verts,Faces, Faces_coeff)

image_ft=normalize_tensor(render_image_from_ftmesh(ftmesh, OTF, box_shape))
t2 = time()

io.imsave("Test_sphere_Hanser.tif",img_as_ubyte(image_ft.detach().cpu().numpy()))
print("forward pass computed successfully in", t2-t1,"seconds", "Image saved as Test_sphere_Hanser.tif")

loss = torch.sum(torch.abs(ftmesh))
t1 = time()
loss.backward()
t2 = time()
print("backward pass computed successfully in", t2-t1,"seconds")


# Gibson-Lanni

In [ ]:
from deltamic import Gibson_Lanni_psf,compute_spatial_frequency_grid,normalize_tensor,render_image_from_ftmesh,Fourier3dMesh,compute_box_size
import trimesh
import numpy as np
from skimage import img_as_ubyte
import torch
import skimage.io as io 
from skimage import img_as_ubyte
from time import time

"""
Parameters:
"""

if torch.cuda.is_available():
    print("Computations done in CUDA!")
    device = 'cuda:0'
    n_subdivisions = 2
    box_shape = np.array([100]*3)
else : 
    print("No CUDA devices available, Computations done in CPU")
    device = 'cpu'
    n_subdivisions = 1
    box_shape = np.array([60]*3).tolist()

"""
Creation of
initial 
configurations: 
"""

mesh = trimesh.primitives.Sphere(subdivisions = n_subdivisions)
verts, faces = np.array(mesh.vertices),np.array(mesh.faces)
verts-=verts.min(axis=0)
verts*=0.5
box_size = compute_box_size(verts, offset = 0.35)
Verts = torch.tensor(verts, dtype = torch.float, device = device,requires_grad = True)
Faces = torch.tensor(faces, dtype = torch.long, device = device)
Faces_coeff = torch.ones(len(Faces),dtype = torch.float, device = device)
box_size = torch.tensor(box_size) 

print("Vertices shape: ",verts.shape,"Faces shape: ", faces.shape)
print("Min/Max x/y/z position of vertices: ",verts.min(axis=0),verts.max(axis=0))
print("box_size: ",box_size, "box_shape",box_shape)

narrowband_thresh = torch.tensor(0,dtype = torch.float, device = device)


psf_params = dict(na=1.27, ni=1.33, wl=0.561, vec_corr="none")

# Set the Nyquist sampling rate
nyquist_sampling = psf_params["wl"] / psf_params["na"] /4

# our oversampling factor, must be odd for easy integration (for peaked symmetrical funtions the kernel must be odd)
oversample_factor = 1

psf_params["res"] = nyquist_sampling / oversample_factor

meshFT = Fourier3dMesh(box_size,box_shape,device=device, dtype = torch.float32)
model = Gibson_Lanni_psf(box_shape,pZ = 0, wavelength =psf_params["wl"], NA = psf_params["na"],ni = psf_params['ni'], 
                         ni0=psf_params['ni'],res_axial=psf_params['res'],res_lateral=psf_params['res'] ,device = device)
PSF = model.forward()
OTF = torch.abs(torch.fft.fftshift(torch.fft.fftn(PSF)))

"""
Image creation:
"""
from deltamic.optimize import VectorAdam

t1 = time()
ftmesh = meshFT(Verts,Faces, Faces_coeff)

image_ft=normalize_tensor(render_image_from_ftmesh(ftmesh, OTF, box_shape))
t2 = time()

io.imsave("Test_sphere.tif",img_as_ubyte(image_ft.detach().cpu().numpy()))
print("forward pass computed successfully in", t2-t1,"seconds", "Image saved as Test_sphere.tif")

loss = torch.sum(torch.abs(ftmesh))
t1 = time()
loss.backward()
t2 = time()
print("backward pass computed successfully in", t2-t1,"seconds")